# 6. Water resources data science

1. Accessing Data with APIs  
   1. APIs basics  
      1. Similarities and differences to snowflake   
      2. Ryan Hankins will post a common dataset  
   2. Accessing APIs with the python “requests” module
   4. Loading API URL paths directly into pandas dataframes   
2. Query OpenET API, get ET data
    3. demonstrate point query and polygon query
    4. get some ET data - both point and poly 
    5. Talk about how OpenET is built on Earth Engine
        6. benefits of earth engine
3. Take ET data over area into a 2d or 3d numpy array 
4. Create pandas dataframe from time series with the point data, with datetime index 
5. Interpolate in two dimensions two subsets of the gridded data
    6. Use Scipy 
       1. Morphology  
       2. Interpolate.griddata
    6. talk about uncertainty 
    7. Plot ET from several points on the same time series the same time series with pandas / seaborn
    8. Determine date of max ET for location
    9. Compare to date of max streamflow?